In [1]:
import numpy as np
import pandas as pd
import os

import torch

os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import sklearn.metrics as sk_metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import random_split, Dataset
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from nam.wrapper import NAMClassifier, MultiTaskNAMClassifier
import skorch
from skorch import NeuralNetClassifier, NeuralNet, NeuralNetBinaryClassifier
import random
seed = 452

In [15]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed)

In [16]:
# 加载数据
cols_A = list(pd.read_csv("/hdd/wmh/other/Cookie_Thieft/feature/"
                        "LIWC-22 Results - all_Control_out - LIWC Analysis.csv").columns)[2:]

cols_B = list(pd.read_csv("/hdd/wmh/other/Cookie_Thieft/feature/pos_tag.csv").columns)[3:]
A = cols_A.copy()

abnormal_dataset = pd.read_csv("/hdd/wmh/other/Cookie_Thieft/feature/merged_abnormal.csv")
normal_dataset = pd.read_csv("/hdd/wmh/other/Cookie_Thieft//feature/merged_normal.csv")

data = normal_dataset
data = pd.concat([abnormal_dataset, normal_dataset])

"""删除embed_ds 特征"""
print(data.shape)
print("================")
for col in cols_A:
    data = data.drop(col, axis=1)

for col in cols_B:
    data = data.drop(col, axis=1)

# try 1
data = data.drop('Feature_4', axis=1)
# try 2 -- 保留
data = data.drop('words_num', axis=1)
# try 3
del_cols = ['Feature_2_high', 'Feature_2_mid-high', 'Feature_2_mid', 'Feature_2_low']
for col in del_cols:
    data = data.drop(col, axis=1)

print(data.shape)
print(list(data.columns))

# binary = ['sex', 'c_charge_degree']
# other = ['age', 'race', 'priors_count', 'length_of_stay']
binary = []
other = list(data.columns)[2:]
# other1 = ['Feature_1_high', 'Feature_1_mid-high', 'Feature_1_mid']# , 'Feature_1_low']
# other2 = ['Feature_2_high', 'Feature_2_mid-high']# , 'Feature_2_mid', 'Feature_2_low']
# other3 = [x for x in ['Feature_2_sink', 'Feature_2_dish', 'Feature_2_mother', 'Feature_2_stool',
#           'Feature_2_boy', 'Feature_2_water', 'Feature_2_jar', 'Feature_2_cookie'] if x in data.columns]

# scaler = MinMaxScaler((-1, 1))
# scaler = Normalizer()
# scaler = StandardScaler()
# data[other] = scaler.fit_transform(data[other])
# data[other] = scaler.fit_transform(data[other])

data_train, data_test = train_test_split(data, train_size=0.9, test_size=0.1, random_state=seed)
# print(data_train)

X_train, y_train = data_train[other + binary], data_train['label']
# X_valid, y_valid = data_valid[other + binary], data_valid['label']
X_test, y_test = data_test[other + binary], data_test['label']

(549, 90)
(549, 30)
['sample', 'label', 'Feature_1_high', 'Feature_1_mid-high', 'Feature_1_mid', 'Feature_1_low', 'Feature_2_mother', 'Feature_2_jar', 'Feature_2_dish', 'Feature_2_cookie', 'Feature_2_water', 'Feature_2_stool', 'Feature_2_boy', 'Feature_2_sink', 'Feature_2_window', 'Feature_2_cabinet', 'Feature_2_cup', 'Feature_2_plant', 'Feature_2_girl', 'Feature_2_curtain', 'Feature_2_floor', 'Feature_3', 'log_embed_ds', 'sentence_num', 'word_per_sentence', 'group1_score', 'group2_score', 'group3_score', 'group4_score', 'avg_clip_score']


In [17]:
data.to_csv("feature.csv", index=False)

In [5]:
model = NAMClassifier(
        num_epochs=300,
        num_learners=5,
        metric='neg_log_loss',
        early_stop_mode='min',
        monitor_loss=True,
        lr=1e-3,
        hidden_sizes=[32, 16],
        batch_size=8,
        val_split=0.1,
        n_jobs=16,
        random_state=42,
        decay_rate=1e-4,
        warm_start=True,
        device='cuda',
    )

In [6]:
model.fit(X_train, y_train)

0     tensor(0.7353, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.6724, device='cuda:0')
1     tensor(0.6414, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.6285, device='cuda:0')
2     tensor(0.6088, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.6240, device='cuda:0')
3     tensor(0.6010, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5886, device='cuda:0')
4     tensor(0.5625, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5450, device='cuda:0')
5     tensor(0.5504, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5499, device='cuda:0')
6     tensor(0.5196, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5265, device='cuda:0')
7     tensor(0.5535, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5395, device='cuda:0')
8     tensor(0.5417, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5374, device='cuda:0')
9     tensor(0.5198, device='cuda:0', grad_fn=<DivBackward0>)     tensor(0.5335, device='cuda:0')
10     tensor(0.5291

In [7]:
# pred = model.predict_proba(X_test)
# sk_metrics.roc_auc_score(y_test, pred)

In [8]:
model = NAMClassifier(
        num_epochs=300,
        num_learners=5,
        metric='neg_log_loss',
        early_stop_mode='min',
        monitor_loss=True,
        lr=1e-3,
        hidden_sizes=[32, 16],
        batch_size=8,
        val_split=0.1,
        n_jobs=16,
        random_state=42,
        decay_rate=1e-4,
        warm_start=True,
        device='cuda',
    )
model.load_checkpoints('/hdd/wmh/other/Cookie_Thieft/nam/output/')

In [9]:
# Evaluate performance of the best model on test set
y_true, y_pred = y_test, model.predict(X_test)
y_prob = model.predict_proba(X_test) 

In [10]:
# Performance Metrics: AUC, ACC, Recall, Precision, F1_score
metrics = [round(roc_auc_score(y_true, y_prob[:]), 4), round(accuracy_score(y_true, y_pred), 4),
           round(recall_score(y_true, y_pred), 4), round(precision_score(y_true, y_pred), 4),
           round(f1_score(y_true, y_pred), 4),
           ]

print('Accuracy metrics')
print('AUC, ACC, Recall, Precision, F1_score')
print(metrics)

Accuracy metrics
AUC, ACC, Recall, Precision, F1_score
[0.9103, 0.8364, 0.8462, 0.9167, 0.88]


In [11]:
# [0.7469, 0.7, 0.8281, 0.7067, 0.7626] num_epochs=1000, max false 0.001

In [12]:
# [0.6075, 0.6, 0.8065, 0.6098, 0.6944]

In [13]:
for m in model.models:
    m.eval() 

In [14]:
import shutil
import os
# if not os.path.exists(f"./output/ours/{seed}/"):
#     os.makedirs(f"./output/ours/{seed}/")

In [15]:
# 移动文件
for i in range(5):
    shutil.copytree(f"./output/{i}/", f"./output/ours/{seed}/{i}")